In [147]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf
import place_cells as pc

def set_groups(df): 
    df['group'] = df['mouse']
    df['group'] = df['group'].replace({'mca24': 'Young Adult', 'mca25': 'Young Adult', 'mca26': 'Young Adult', 'mca27': 'Young Adult', 
                                       'mca28': 'Middle Age', 'mca29': 'Middle Age', 'mca30': 'Middle Age', 'mca31': 'Middle Age', 
                                       'mca32': 'Aged', 'mca33': 'Aged', 'mca34': 'Aged', 'mca35': 'Aged', 
                                       'mca36': 'Young Adult', 'mca37': 'Young Adult', 'mca38': 'Young Adult', 'mca39': 'Young Adult', 
                                       'mca40': 'Middle Age', 'mca41': 'Middle Age', 'mca42': 'Middle Age', 'mca43': 'Middle Age',
                                       'mca44': 'Aged', 'mca45': 'Aged', 'mca46': 'Aged', 'mca47': 'Aged'})
    return df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### The code below can be used to visualize results after each day.

In [ ]:
## Set path variables
behavior_type = 'circletrack_data'
csv_name = 'circle_track.csv'
project_dir = 'MultiCon_Behavior'
experiment_dir = 'MultiCon_Aging2'
dpath = ''
dpath = f'../../../{project_dir}/{experiment_dir}/{behavior_type}/data/**/**/**/{csv_name}'
fig_path = (f'../../../{project_dir}/{experiment_dir}/intermediate_figures/')
group_colors = ['red', 'midnightblue', 'darkgrey']
group_colors_dict = {'Young Adult': 'darkgrey', 'Middle Age': 'midnightblue', 'Aged': 'red',
                     'Young Adult Thresh3': 'turquoise', 'Young Adult Thresh5': 'midnightblue'}
chance_color = 'darkgrey'

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

## Create file list dataframe
file_list = ctb.get_file_list(dpath)
mouse_id = []
for file in file_list:
    mouse_id.append(ctb.get_mouse(file, str2match='(mca[0-9]+)'))
combined = ctb.combine(file_list, mouse_id)

In [ ]:
lick_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    lick_data = ctb.get_lick_accuracy(combined, mouse)
    lick_df = pd.concat([lick_df, lick_data], ignore_index=True)
lick_df = set_groups(lick_df)
## Lick accuracy plot
fig = pf.plot_behavior_across_days(lick_df, x_var='day', y_var='percent_correct', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey', plot_datapoints=True,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track: Lick Accuracy'], height=700, width=700)
fig.show()
# fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))
# lick_df.mixed_anova(dv='percent_correct', subject='mouse', within='day', between='group')

In [ ]:
reward_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    reward_data = ctb.get_total_rewards(combined, mouse)
    reward_df = pd.concat([reward_df, reward_data], ignore_index=True)
reward_df = set_groups(reward_df)
## Total rewards plot
fig = pf.plot_behavior_across_days(reward_df, x_var='day', y_var='total_rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey', expert_line=False, chance=False, plot_datapoints=True,
                                   x_title='Day', y_title='Total Rewards', titles=['Circle Track: Rewards'], height=700, width=700)
fig.show()
# fig.write_image(pjoin(fig_path, 'total_rewards.png'))

### Set up run order for the day.

In [ ]:
day = 'Day 38' ## edit depending on day
retired_mice = [] ## add retired mouse to this list
maze_info = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/maze_info.csv')
todays_mazes = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/{experiment_dir} - TodaysMazes.csv') ## download at end of the day
run_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
letter_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
for mouse in np.unique(todays_mazes['Mouse']):
    if mouse in retired_mice:
        pass
    else:
        maze_letter = todays_mazes.loc[todays_mazes['Mouse'] == mouse, day].reset_index(drop=True)

        if 'A' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'A'"].reset_index(drop=True)
        elif 'B' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'B'"].reset_index(drop=True)
        elif 'C' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'C'"].reset_index(drop=True)
        elif 'D' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'D'"].reset_index(drop=True)

        if maze_number[0] == 1:
            run_dict['maze1'].append(mouse)
            letter_dict['maze1'].append(maze_letter[0])
        elif maze_number[0] == 2:
            run_dict['maze2'].append(mouse)
            letter_dict['maze2'].append(maze_letter[0])
        elif maze_number[0] == 3:
            run_dict['maze3'].append(mouse)
            letter_dict['maze3'].append(maze_letter[0])
        elif maze_number[0] == 4:
            run_dict['maze4'].append(mouse)
            letter_dict['maze4'].append(maze_letter[0])
## Convert to dataframe to paste into excel         
run_df = pd.DataFrame.from_dict(run_dict, orient='index').T 
letter_df = pd.DataFrame.from_dict(letter_dict, orient='index').T

In [ ]:
run_df

In [ ]:
letter_df

### Analyze preprocessed behavior data.

In [148]:
## Settings
project_dir = 'MultiCon_Behavior'
experiment_dir = 'MultiCon_Aging2'
lin_path = f'../../../{project_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{project_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{project_dir}/{experiment_dir}/intermediate_figures'
young_adult = [f'mca{x}' for x in np.arange(24, 28)] + [f'mca{x}' for x in np.arange(36, 40)]
middle_aged = [f'mca{x}' for x in np.arange(28, 32)] + [f'mca{x}' for x in np.arange(40, 44)]
aged = [f'mca{x}' for x in np.arange(32, 36)] + [f'mca{x}' for x in np.arange(44, 48)]
three_color_plots = ['red', 'midnightblue', 'darkgrey']
group_colors_dict = {'Young Adult': 'darkgrey', 'Middle Aged': 'midnightblue', 'Aged': 'red',
                     'Young Adult Thresh1': 'darkgrey', 'Young Adult Thresh5': 'black',
                     'Middle Aged Thresh1': 'turquoise', 'Middle Aged Thresh5': 'midnightblue',
                     'Aged Thresh1': 'red', 'Aged Thresh5': 'darkred'}
context_colors = ['midnightblue', 'red', 'darkorchid', 'forestgreen']
chance_color = 'darkgrey'
bin_size = 4 ## for binning across trials
error_colors = ['rgba(169,169,169,0.4)', 'rgba(169,169,169,0.4)', 'rgba(169,169,169,0.4)'] ## rgba is the only way to get the error band transparent
opacity = 0.8 ## for binning across trials

### Linear track results.

In [150]:
## Linear track rewards
result_dict = {'mouse': [], 'day': [], 'group': [], 'rewards': []}
for mouse in os.listdir(lin_path)[:-1]:
    mouse_path = pjoin(lin_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        result_dict['mouse'].append(mouse)
        result_dict['day'].append(idx+1)
        result_dict['group'].append(group)
        result_dict['rewards'].append(np.sum(behav['water']))
results_df = pd.DataFrame(result_dict)

In [152]:
## Plot rewards across days in linear track
fig = pf.plot_behavior_across_days(results_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=False, plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
results_df.mixed_anova(dv='rewards', within='day', subject='mouse', between='group')
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))
results_df.mixed_anova(dv='rewards', subject='mouse', within='day', between='group')

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,group,1030.416667,2,21,515.208333,1.195329,3.223716e-01,NaN,0.102206,NaN,NaN,NaN,NaN
1,day,28181.050000,4,84,7045.262500,145.102869,7.138342e-37,2.416855e-23,0.873572,0.630424,False,0.308521,0.002851
2,Interaction,1317.250000,8,84,164.656250,3.391228,2.023328e-03,NaN,0.244127,NaN,NaN,NaN,NaN


### Analyze circle track lick accuracy.

In [24]:
## Looking at first licks
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh5': [], 'rewards': []}
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
            behav_data = behav[~behav['probe']]
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
            pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
            circletrack_results['mouse'].append(mouse)
            circletrack_results['day'].append(idx+1)
            circletrack_results['group'].append(group)
            circletrack_results['session'].append(np.unique(behav['session'])[0])
            circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
            circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
            circletrack_results['rewards'].append(np.sum(behav_data['water']))
ct_df = pd.DataFrame(circletrack_results)

In [27]:
## Plot lick accuracy across days for thresh of 1
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh1', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=True, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_thresh1.png'))
# ct_df.mixed_anova(dv='lick_accuracy_thresh1', within='day', subject='mouse', between='group')

In [29]:
## Plot lick accuracy across days for thresh of 5
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh5', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=True, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_thresh5.png'))
# ct_df.mixed_anova(dv='lick_accuracy_thresh5', within='day', subject='mouse', between='group')

In [ ]:
## Look at lick accuracy just when a mouse is in A.
y_var = 'lick_accuracy_thresh5'
a_data = ct_df[ct_df['session'] == 'A']
fig = pf.plot_behavior_across_days(a_data, x_var='day', y_var=y_var, groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, f'{y_var}_in_A.png'))
a_data.mixed_anova(dv=y_var, within='day', subject='mouse', between='group')

In [ ]:
## Look at lick accuracy just when a mouse is in A.
y_var = 'rewards'
a_data = ct_df[ct_df['session'] == 'A']
fig = pf.plot_behavior_across_days(a_data, x_var='day', y_var=y_var, groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, f'{y_var}_in_A.png'))
a_data.mixed_anova(dv=y_var, within='day', subject='mouse', between='group')

### Look at trial by trial differences in lick accuracy for the first or fifth lick.

In [ ]:
## Calculate for lick accuracy across trials
data = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'trial_acc_1': [], 'trial_acc_5': []}
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    day_counter = 1
    session_id = np.nan
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
        reward_one, reward_two = np.unique(behav_data['reward_one'])[0], np.unique(behav_data['reward_two'])[0]
        pc_1 = ctb.lick_accuracy(behav_data, reward_one, reward_two, lick_threshold=1, by_trials=True)
        pc_5 = ctb.lick_accuracy(behav_data, reward_one, reward_two, lick_threshold=5, by_trials=True)
        binned_pc_1 = ctb.bin_data(pc_1, bin_size=bin_size)
        binned_pc_5 = ctb.bin_data(pc_5, bin_size=bin_size)
        data['mouse'].append(mouse)
        data['day'].append(idx+1)
        data['group'].append(group)
        data['trial_acc_1'].append(binned_pc_1)
        data['trial_acc_5'].append(binned_pc_5)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
trial_acc = pd.DataFrame(data)

In [ ]:
group_data = {'Young Adult': [], 'Middle Aged': [], 'Aged': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult'].append(np.nanstd(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged'].append(np.nanstd(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
group_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy first lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=2, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(group_df['day']))+1)])
for idx, day in enumerate(np.unique(group_df['day'])):
    plot_data = group_df[group_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=2)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_across_groups.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_groups.png'))

In [ ]:
group_data = {'Young Adult': [], 'Middle Aged': [], 'Aged': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
group_df_five = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(group_df_five['day']))+1)])
for idx, day in enumerate(np.unique(group_df_five['day'])):
    plot_data = group_df_five[group_df_five['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
# if only_min_trials:
#     fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_across_groups.png'))
# else:
#     fig.write_image(pjoin(fig_path, 'lick_acc_across_groups.png'))

### Look at trial by trial changes in lick accuracy comparing threshold 1 vs 5 for each group.

In [ ]:
group_of_interest = 'Young Adult'
group_data = {'Young Adult Thresh1': [], 'Young Adult Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Young Adult Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
young_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(young_df['day']))+1)])
for idx, day in enumerate(np.unique(young_df['day'])):
    plot_data = young_df[young_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_young.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_young.png'))

In [ ]:
group_of_interest = 'Middle Aged'
group_data = {'Middle Aged Thresh1': [], 'Middle Aged Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Middle Aged Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
middle_aged_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(middle_aged_df['day']))+1)])
for idx, day in enumerate(np.unique(middle_aged_df['day'])):
    plot_data = middle_aged_df[middle_aged_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_middle_aged.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_middle_aged.png'))

In [ ]:
group_of_interest = 'Aged'
group_data = {'Aged Thresh1': [], 'Aged Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Aged Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
aged_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(aged_df['day']))+1)])
for idx, day in enumerate(np.unique(aged_df['day'])):
    plot_data = aged_df[aged_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_aged.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_aged.png'))

### Look at probe accuracy.

In [153]:
## Looking at first licks
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh5': []}
for mouse in os.listdir(circle_path)[:-1]: ## somehow a .DS_Store appeared????????
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
            if any(behav['probe']):
                behav = behav[behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
                pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
                circletrack_results['mouse'].append(mouse)
                circletrack_results['day'].append(idx+1)
                circletrack_results['group'].append(group)
                circletrack_results['session'].append(np.unique(behav['session'])[0])
                circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
                circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
y_variable = 'lick_accuracy_thresh1'
sess_pairs = [('A', 'AP'), ('B', 'BP'), ('C', 'CP'), ('D', 'DP')]
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, titles=['A', 'B', 'C', 'D'], 
                               width=1000, shared_y=True)
for idx, pair in enumerate(sess_pairs):
    loop_data = ct_df[(ct_df['session'] == pair[0]) | (ct_df['session'] == pair[1])]
    loop_data = loop_data.replace({pair[0]: 'First', pair[1]: 'Last'})
    avg_data = loop_data.groupby(['session', 'group'], as_index=False).agg({y_variable: ['mean', 'sem']})
    for group in np.unique(avg_data['group']):
        gdata = avg_data[avg_data['group'] == group]
        fig.add_trace(go.Scatter(x=gdata['session'], y=gdata[y_variable]['mean'], name=group,
                                 marker_color=group_colors_dict[group], showlegend=False, legendgroup=group,
                                 error_y=dict(type='data', array=gdata[y_variable]['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig['data'][2]['showlegend'] = True
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'thresh1_probe_acc.png'))

In [154]:
y_variable = 'lick_accuracy_thresh5'
sess_pairs = [('A', 'AP'), ('B', 'BP'), ('C', 'CP'), ('D', 'DP')]
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, titles=['A', 'B', 'C', 'D'], 
                               width=1000, shared_y=True)
for idx, pair in enumerate(sess_pairs):
    loop_data = ct_df[(ct_df['session'] == pair[0]) | (ct_df['session'] == pair[1])]
    loop_data = loop_data.replace({pair[0]: 'First', pair[1]: 'Last'})
    avg_data = loop_data.groupby(['session', 'group'], as_index=False).agg({y_variable: ['mean', 'sem']})
    for group in np.unique(avg_data['group']):
        gdata = avg_data[avg_data['group'] == group]
        fig.add_trace(go.Scatter(x=gdata['session'], y=gdata[y_variable]['mean'], name=group,
                                 marker_color=group_colors_dict[group], showlegend=False, legendgroup=group,
                                 error_y=dict(type='data', array=gdata[y_variable]['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig['data'][2]['showlegend'] = True
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'thresh5_probe_acc.png'))

### Days to Criteria

In [78]:
todays_mazes = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/{experiment_dir} - TodaysMazes.csv')
melt_df = todays_mazes.melt(id_vars='Mouse')
finished_mice = np.unique(melt_df['Mouse'][melt_df['value'] == 'DONE'])
mdata = pd.DataFrame()
for mouse in finished_mice:
    loop_data = todays_mazes[todays_mazes['Mouse'] == mouse]
    mdata = pd.concat([mdata, loop_data])
melt_df = mdata.melt(id_vars='Mouse')
melt_df = melt_df[~pd.isna(melt_df['value'])]
melt_df = melt_df[(melt_df['value'] != 'AP') & (melt_df['value'] != 'BP') & (melt_df['value'] != 'CP') & (melt_df['value'] != 'DP') & (melt_df['value'] != 'DONE') & (melt_df['value'] != 'CT') & (melt_df['value'] != 'DT')]

In [79]:
days_to = melt_df.groupby(['Mouse', 'value'], as_index=False).agg({'value': 'count'})
mouse_days_to = pd.DataFrame()
for mouse in np.unique(days_to['Mouse']):
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    loop_data = days_to[days_to['Mouse'] == mouse]
    loop_data.loc[:, 'context'] = ['A', 'B', 'C', 'D']
    loop_data.loc[:, 'group'] = group
    mouse_days_to = pd.concat([mouse_days_to, loop_data])
nonA_data = mouse_days_to[mouse_days_to['context'] != 'A']

/tmp/ipykernel_56185/523038100.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56185/523038100.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_56185/523038100.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_5618

In [80]:
nonA_data = mouse_days_to[mouse_days_to['context'] != 'A']
fig = pf.custom_graph_template(x_title='', y_title='Days to Criteria')
for group in np.unique(nonA_data['group']):
    gdata = nonA_data[nonA_data['group'] == group]
    avg_data = gdata.groupby(['context'], as_index=False).agg({'value': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_data['context'], y=avg_data['value']['mean'], name=group, marker_color=group_colors_dict[group],
                             error_y=dict(type='data', array=avg_data['value']['sem'])))
# fig.update_yaxes(range=[0, 6.5])
fig.show()
fig.write_image(pjoin(fig_path, 'days_to_criteria.png'))

### Create a velocity distribution for all three groups.

In [ ]:
young_v = []
middle_aged_v = []
aged_v = []
downsample_factor = 42
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        x_cm, y_cm = ctb.convert_to_cm(x=behav['x'], y=behav['y'], update_dataframe=False)
        velocity, running = pc.define_running_epochs(x_cm[::downsample_factor], y_cm[::downsample_factor], behav['t'][::downsample_factor])
        avg_vel = np.mean(velocity)
        if group == 'Young Adult':
            young_v.append(avg_vel)
        elif group == 'Middle Aged':
            middle_aged_v.append(avg_vel)
        else:
            aged_v.append(avg_vel)

In [ ]:
fig = pf.custom_graph_template(x_title='Average Velocity (cm/s)', y_title='Probability', height=500, width=500)
fig.add_trace(go.Histogram(x=young_v, marker_color=group_colors_dict['Young Adult'], 
                           name='Young Adult', opacity=0.6, histnorm='probability'))
fig.add_trace(go.Histogram(x=middle_aged_v, marker_color=group_colors_dict['Middle Aged'], 
                           name='Middle Aged', opacity=0.6, histnorm='probability'))
fig.add_trace(go.Histogram(x=aged_v, marker_color=group_colors_dict['Aged'], 
                           name='Aged', opacity=0.6, histnorm='probability'))
fig.update_layout(barmode='overlay')
fig.show()
fig.write_image(pjoin(fig_path, 'velocity_distribution.png'))

In [ ]:
young_v = np.asarray(young_v)[~np.isnan(young_v)]
anova_df = pd.DataFrame({'young': young_v,
                         'middle_aged': middle_aged_v[0:len(young_v)],
                         'aged': aged_v[0:len(young_v)]})
anova_melt = anova_df.melt(value_vars=['young', 'middle_aged', 'aged'], value_name='velocity', var_name='group')
print(pg.homoscedasticity(anova_melt, dv='velocity', group='group'))
anova_melt.welch_anova(dv='velocity', between='group')

In [ ]:
pg.pairwise_gameshowell(anova_melt, dv='velocity', between='group')

### Separate velocity distributions by days.

In [ ]:
vel_dict = {'mouse': [], 'group': [], 'day': [], 'context_day': [], 'session': [], 'avg_velocity': [], 'max_velocity': [], 'vel_values': []}
downsample_factor = 42
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        x_cm, y_cm = ctb.convert_to_cm(x=behav['x'], y=behav['y'], update_dataframe=False)
        velocity, running = pc.define_running_epochs(x_cm[::downsample_factor], y_cm[::downsample_factor], behav['t'][::downsample_factor])
        vel_dict['mouse'].append(mouse)
        vel_dict['group'].append(group)
        vel_dict['day'].append(idx+1)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            vel_dict['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            vel_dict['context_day'].append(day_counter)

        vel_dict['session'].append(np.unique(behav['session'])[0])
        vel_dict['avg_velocity'].append(np.mean(velocity))
        vel_dict['max_velocity'].append(np.max(velocity))
        vel_dict['vel_values'].append(velocity)
vel = pd.DataFrame(vel_dict)

In [ ]:
## Plot average velocity for each group across days in each context
session_list = ['A', 'B', 'C', 'D']
vel_contexts = pd.DataFrame()
for session in session_list:
    loop_data = vel[vel['session'] == session]
    vel_contexts = pd.concat([vel_contexts, loop_data], ignore_index=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, width=1000, 
                               shared_y=True, shared_x=True, titles=session_list)
for group in vel_contexts['group'].unique():
    gdata = vel_contexts[vel_contexts['group'] == group].groupby(['context_day', 'session'], as_index=False).agg({'avg_velocity': ['mean', 'sem']})
    for idx, sess in enumerate(gdata['session'].unique()):
        sess_data = gdata[gdata['session'] == sess]
        fig.add_trace(go.Scatter(x=sess_data['context_day'], y=sess_data['avg_velocity']['mean'], name=group,
                                 line_color=group_colors_dict[group], legendgroup=group, showlegend=False,
                                 error_y=dict(type='data', array=sess_data['avg_velocity']['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][5]['showlegend'] = True
fig['data'][11]['showlegend'] = True
fig.update_xaxes(title='Day in Context')
fig.update_yaxes(title='Average Velocity (cm/s)')
fig.show()

### Lick accuracy heatmap.

In [ ]:
## Looking at first licks
lick_thresholds = [1, 2, 3, 4, 5]
circletrack_results = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh2': [],
                       'lick_accuracy_thresh3': [], 'lick_accuracy_thresh4': [], 'lick_accuracy_thresh5': []}
for mouse in os.listdir(circle_path)[:-1]: ##.DS_store
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['group'].append(group)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        for thresh in lick_thresholds:
            lick_accuracy = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=thresh, by_trials=False)
            circletrack_results[f'lick_accuracy_thresh{thresh}'].append(lick_accuracy)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
ct_df = pd.DataFrame(circletrack_results)
## Average for each lick accuracy threshold, excluding final probe days
select_sessions = ct_df[(ct_df['session'] != 'AP') & (ct_df['session'] != 'BP') & (ct_df['session'] != 'CP') & (ct_df['session'] != 'DP')]
avg_data = select_sessions.groupby(['context_day', 'group', 'session'], as_index=False).agg('mean')

In [ ]:
## Young adult mice
young_acc = avg_data[avg_data['group'] == 'Young Adult'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(young_acc['session'])):
    sess_data = young_acc[young_acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
## Middle aged mice
acc = avg_data[avg_data['group'] == 'Middle Aged'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(acc['session'])):
    sess_data = acc[acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
acc = avg_data[avg_data['group'] == 'Aged'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(acc['session'])):
    sess_data = acc[acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
fig = pf.custom_graph_template(x_title='', y_title='', rows=3, columns=4, shared_y=True, shared_x=True,
                               height=1000, width=1200, titles=['A', 'B', 'C', 'D'])
for g_idx, group in enumerate(['Young Adult', 'Middle Aged', 'Aged']):
    acc = avg_data[avg_data['group'] == group].reset_index(drop=True)
    for sess_idx, session in enumerate(np.unique(acc['session'])):
        sess_data = acc[acc['session'] == session].reset_index(drop=True)
        arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
        arr.fill(np.nan)
        for idx, col in enumerate(sess_data.columns[4:]):
            arr[idx] = sess_data[col].to_numpy()
        x_axis = np.arange(1, arr.shape[1]+1)
        fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                                coloraxis='coloraxis'), row=g_idx+1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=3)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_heatmaps.png'))

### Reminder day for IHC

In [ ]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 
                       'lick_accuracy_thresh5': [], 'rewards': []}
for mouse in os.listdir(circle_path)[:-1]: ## somehow a .DS_Store appeared????????
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        if (np.unique(behav['session'])[0] != 'DT' ) & (np.unique(behav['session'])[0] != 'CT' ):
            pass
        else:
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
            pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
            circletrack_results['mouse'].append(mouse)
            circletrack_results['day'].append(idx+1)
            circletrack_results['group'].append(group)
            circletrack_results['session'].append(np.unique(behav['session'])[0])
            circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
            circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
            circletrack_results['rewards'].append(np.sum(behav['water']))
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot lick accuracy thresh5 for all groups
var = 'lick_accuracy_thresh5'
avg_data = ct_df.groupby(['group'], as_index=False).agg({var: ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)')
for group in np.unique(avg_data['group']):
    gdata = avg_data[avg_data['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['group'], y=gdata[var]['mean'], mode='markers', name=group, showlegend=False,
                            marker_color=group_colors_dict[group], error_y=dict(type='data', array=gdata[var]['sem']),
                            marker_size=5))
for mouse in np.unique(ct_df['mouse']):
    mdata = ct_df[ct_df['mouse'] == mouse]
    group = mdata['group'].values[0]
    fig.add_trace(go.Scatter(x=mdata['group'], y=mdata[var], mode='markers', name=group, showlegend=False,
                             marker_color=group_colors_dict[group]))

fig.show()

In [ ]:
## Plot rewards for all groups
var = 'rewards'
avg_data = ct_df.groupby(['group'], as_index=False).agg({var: ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Rewards')
for group in np.unique(avg_data['group']):
    gdata = avg_data[avg_data['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['group'], y=gdata[var]['mean'], mode='markers', name=group, showlegend=False,
                            marker_color=group_colors_dict[group], error_y=dict(type='data', array=gdata[var]['sem'])))
for mouse in np.unique(ct_df['mouse']):
    mdata = ct_df[ct_df['mouse'] == mouse]
    group = mdata['group'].values[0]
    fig.add_trace(go.Scatter(x=mdata['group'], y=mdata[var], mode='markers', name=group, showlegend=False,
                             marker_color=group_colors_dict[group]))

fig.show()

### Plotting lick accuracy in each context across days, where each group is on one subplot.

In [4]:
## Looking at 5th lick
circletrack_results = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'session': [], 'lick_accuracy': []}
for mouse in os.listdir(circle_path)[:-1]: ##.DS_store
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['group'].append(group)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        lick_accuracy = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
        circletrack_results['lick_accuracy'].append(lick_accuracy)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
ct_df = pd.DataFrame(circletrack_results)

In [22]:
context_colors = ['midnightblue', 'red', 'darkorchid', 'forestgreen']
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=3, width=900,
                               titles=['Young Adult', 'Middle Aged', 'Aged'], shared_y=True, shared_x=True)
session_list = ['A', 'B', 'C', 'D']
acc_contexts = pd.DataFrame()
for session in session_list:
    loop_data = ct_df[ct_df['session'] == session]
    acc_contexts = pd.concat([acc_contexts, loop_data], ignore_index=True)
grouped_data = acc_contexts.groupby(['group', 'context_day', 'session'], as_index=False).agg({'lick_accuracy': ['mean', 'sem']})

for idx, group in enumerate(reversed(grouped_data['group'].unique())):
    gdata = grouped_data[grouped_data['group'] == group]
    for sess_idx, session in enumerate(gdata['session'].unique()):
        plot_data = gdata[gdata['session'] == session]
        fig.add_trace(go.Scatter(x=plot_data['context_day'], y=plot_data['lick_accuracy']['mean'],
                                 name=session, line_color=context_colors[sess_idx], legendgroup=session, showlegend=False,
                                 error_y=dict(type='data', array=plot_data['lick_accuracy']['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig['data'][2]['showlegend'] = True
fig['data'][3]['showlegend'] = True
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1, range=[0, 100])
fig.update_xaxes(title='Day in Context')
fig.show()

### Look at the density of licks around rewarded and non-rewarded ports.

In [156]:
lick_dict = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'session': [], 'reward_ports': [],
             'front_ports': [], 'back_ports': [], 'final_ports': []}
for mouse in os.listdir(circle_path)[:-1]: ##.DS_store
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        reward_one, reward_two = behav['reward_one'].unique()[0], behav['reward_two'].unique()[0]
        front_ports, back_ports = ctb.front_back_ports(reward_list=[reward_one, reward_two])

        lick_data = behav[behav['lick_port'] != -1]
        port_licks = lick_data.groupby(['lick_port'], as_index=False).agg({'lick_port': 'count'})
        total_licks = lick_data.agg({'lick_port': 'count'})['lick_port']
        reward_port_licks = 0
        front_port_licks = 0
        back_port_licks = 0
        final_port_licks = 0
        for _, row in port_licks.iterrows():
            p_num = row.name + 1 ## since name starts at 0, and our reward ports start at 1
            if p_num in [reward_one, reward_two]:
                reward_port_licks = reward_port_licks + row['lick_port']
            elif p_num in front_ports:
                front_port_licks = front_port_licks + row['lick_port']
            elif p_num in back_ports:
                back_port_licks = back_port_licks + row['lick_port']
            else:
                final_port_licks = final_port_licks + row['lick_port']

        lick_dict['mouse'].append(mouse)
        lick_dict['day'].append(idx+1)
        lick_dict['group'].append(group)
        lick_dict['session'].append(behav['session'].unique()[0])
        lick_dict['reward_ports'].append((reward_port_licks / total_licks) * 100)
        lick_dict['front_ports'].append((front_port_licks / total_licks) * 100)
        lick_dict['back_ports'].append((back_port_licks / total_licks) * 100)
        lick_dict['final_ports'].append((final_port_licks / total_licks) * 100)

        if behav['session'].unique()[0] != session_id:
            day_counter = 1
            session_id = behav['session'].unique()[0]
            lick_dict['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = behav['session'].unique()[0]
            lick_dict['context_day'].append(day_counter)
lick_df = pd.DataFrame(lick_dict)
avg_licks = lick_df.groupby(['context_day', 'group', 'session'], as_index=False).agg({'reward_ports': ['mean', 'sem'], 'front_ports': ['mean', 'sem'],
                                                                                      'back_ports': ['mean', 'sem'], 'final_ports': ['mean', 'sem']})

/tmp/ipykernel_56185/3372198452.py:36: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:37: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:38: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:39: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:36: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:37: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:38: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:39: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:36: RuntimeWarning:

invalid value encountered in scalar divide

/tmp/ipykernel_56185/3372198452.py:37: RuntimeWarning:

invalid value encountered in scalar divide



In [171]:
lick_df

,mouse,day,context_day,group,session,reward_ports,front_ports,back_ports,final_ports
0,mca24,1,1,Young Adult,A,36.346863,28.413284,16.605166,18.634686
1,mca24,2,2,Young Adult,A,39.925174,24.318546,21.378942,14.377338
2,mca24,3,3,Young Adult,A,62.253695,25.800493,8.189655,3.756158
3,mca24,4,4,Young Adult,A,82.197802,11.208791,3.956044,2.637363
4,mca24,5,5,Young Adult,A,86.325678,8.246347,2.609603,2.818372
...,...,...,...,...,...,...,...,...,...
665,mca47,25,2,Aged,D,65.149007,18.211921,7.781457,8.857616
666,mca47,26,3,Aged,D,82.854209,12.217659,1.437372,3.490760
667,mca47,27,4,Aged,D,71.840796,17.213930,2.189055,8.756219
668,mca47,28,1,Aged,DP,49.212925,12.593206,16.901408,21.292461


In [188]:
## Young mice in A
age_group = 'Young Adult'
context = 'A'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=3, columns=4, shared_y=True, height=800, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3
    else:
        row, col = 3, idx - 7

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        else:
            row, col = 3, idx - 7
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

In [189]:
## Middle aged mice in A
age_group = 'Middle Aged'
context = 'A'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=3, columns=4, shared_y=True, height=800, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3
    else:
        row, col = 3, idx - 7

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        else:
            row, col = 3, idx - 7
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

In [190]:
## Aged mice in A
age_group = 'Aged'
context = 'A'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=3, columns=4, shared_y=True, height=800, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3
    else:
        row, col = 3, idx - 7

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        else:
            row, col = 3, idx - 7
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

In [193]:
## Young mice in B
age_group = 'Young Adult'
context = 'B'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=2, columns=4, shared_y=True, height=600, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3
    else:
        row, col = 3, idx - 7

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

In [196]:
## Middle aged mice in B
age_group = 'Middle Aged'
context = 'B'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=2, columns=4, shared_y=True, height=600, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

In [203]:
## Aged mice in B
age_group = 'Aged'
context = 'B'
x_axis = ['RP', 'FP', 'BP', 'LP']
licks = avg_licks[(avg_licks['group'] == age_group) & (avg_licks['session'] == context)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=4, columns=4, shared_y=True, height=1000, width=800,
                               titles=[f'Day {x}' for x in np.arange(1, licks.shape[0]+1)])
for idx, day in enumerate(licks['context_day'].unique()):
    if idx < 4:
        row, col = 1, idx + 1
    elif (idx >= 4) & (idx < 8):
        row, col = 2, idx - 3
    elif (idx >= 8) & (idx < 12):
        row, col = 3, idx - 7
    else:
        row, col = 4, idx - 11

    pdata = licks[licks['context_day'] == day]
    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=group_colors_dict[age_group],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)

sub_df = lick_df[(lick_df['session'] == context) & (lick_df['group'] == age_group)]
for mouse in sub_df['mouse'].unique():
    mdata = sub_df[sub_df['mouse'] == mouse]
    for idx, day in enumerate(mdata['context_day'].unique()):
        if idx < 4:
            row, col = 1, idx + 1
        elif (idx >= 4) & (idx < 8):
            row, col = 2, idx - 3
        elif (idx >= 8) & (idx < 12):
            row, col = 3, idx - 7
        else:
            row, col = 4, idx - 11
        
        data = mdata[mdata['context_day'] == day]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color=group_colors_dict[age_group], marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='Percent of Licks', col=1, dtick=25, range=[0, 100])
fig.show()

### Determine where 1st, 2nd, 3rd, 4th, and 5th licks are occurring.